# Regex Field Extraction from Agent-Customer Conversations (FINAL VERSION)

This notebook extracts structured data (email, phone, zip_code, order_id) from unstructured conversation text using comprehensive regex patterns with significantly improved order ID detection.


In [1]:
import pandas as pd
import re
import os


## Function Definitions

Each function extracts a specific field from the conversation text using comprehensive regex patterns.


In [2]:
def extract_email(text):
    """
    Extract email address from conversation text.

    Args:
        text: String containing conversation text

    Returns:
        Email address if found, "NA" otherwise
    """
    if pd.isna(text) or text == "":
        return "NA"

    # Pattern to match email addresses
    email_pattern = r'[\w\.-]+@[\w\.-]+'
    match = re.search(email_pattern, str(text))

    if match:
        return match.group(0)
    else:
        return "NA"


In [3]:
def extract_phone(text):
    """
    Extract phone number from conversation text.
    Phone numbers MUST have parentheses around area code: (XXX) XXX-XXXX
    This strict pattern prevents confusion with order IDs.

    Args:
        text: String containing conversation text

    Returns:
        Phone number if found, "NA" otherwise
    """
    if pd.isna(text) or text == "":
        return "NA"

    text_str = str(text)

    # STRICT phone number patterns - must have parentheses or explicit phone formatting
    # Only match formats like: (003) 941-7614, (752) 693-4642, etc.
    phone_patterns = [
        r'\((\d{3})\)\s*(\d{3})[-.\s]?(\d{4})',  # (003) 941-7614 or (003) 941 7614 or (003)9417614
        r'\((\d{3})\)\s*(\d{7})',  # (003) 9417614 (no space/dash)
    ]

    for pattern in phone_patterns:
        matches = list(re.finditer(pattern, text_str))
        for match in matches:
            # Check context - make sure it's not preceded by "order id" or similar
            start_pos = match.start()
            context_before = text_str[max(0, start_pos-50):start_pos].lower()

            # Skip if it's in order ID context
            if re.search(r'order\s+(id|number|#)', context_before):
                continue

            # Reconstruct phone number
            if len(match.groups()) == 3:
                area_code = match.group(1)
                first_part = match.group(2)
                second_part = match.group(3)
                phone_clean = area_code + first_part + second_part
            elif len(match.groups()) == 2:
                area_code = match.group(1)
                rest = match.group(2)
                phone_clean = area_code + rest
            else:
                phone_clean = ''.join(match.groups())

            # Verify it's 10 digits
            if phone_clean.isdigit() and len(phone_clean) == 10:
                return f"{phone_clean[:3]}-{phone_clean[3:6]}-{phone_clean[6:]}"

    return "NA"


In [4]:
def extract_zip_code(text):
    """
    Extract zip code from conversation text.
    Supports both 5-digit and 5+4 format (e.g., 12345 or 12345-6789).
    Improved to catch zip codes mentioned explicitly or in address contexts.

    Args:
        text: String containing conversation text

    Returns:
        Zip code if found, "NA" otherwise
    """
    if pd.isna(text) or text == "":
        return "NA"

    text_str = str(text)

    # First, look for explicit mentions of zip code
    # Pattern 1: "zip code" or "zip" followed by a 5-digit number
    explicit_zip_patterns = [
        r'(?:zip\s*(?:code|is|:)?\s*)(\d{5}(?:-\d{4})?)',  # "zip code 12345" or "zip: 12345" or "zip is 12345"
        r'(?:zip\s+)(\d{5}(?:-\d{4})?)',  # "zip 12345"
    ]

    for pattern in explicit_zip_patterns:
        match = re.search(pattern, text_str, re.IGNORECASE)
        if match:
            zip_code = match.group(1)
            return zip_code

    # Pattern 2: 5+4 format (12345-6789) - always return if found
    zip_plus4_pattern = r'\b(\d{5}-\d{4})\b'
    match = re.search(zip_plus4_pattern, text_str)
    if match:
        return match.group(1)

    # Pattern 3: Look for 5-digit numbers in address contexts
    # Address pattern: Street, City, State ZIP
    address_zip_pattern = r'(?:street|ave|avenue|road|rd|drive|dr|blvd|boulevard|way|ln|lane|st|circle)\s+[^,]*,\s*[^,]*,\s*[A-Z]{2}\s+(\d{5})'
    match = re.search(address_zip_pattern, text_str, re.IGNORECASE)
    if match:
        return match.group(1)

    # Pattern 4: Look for standalone 5-digit numbers that are likely zip codes
    # Check all 5-digit numbers and determine if they're zip codes
    all_5digit = list(re.finditer(r'\b(\d{5})\b', text_str))

    for match in all_5digit:
        zip_candidate = match.group(1)
        start_pos = match.start()
        end_pos = match.end()

        # Get context around the number
        context_before = text_str[max(0, start_pos-50):start_pos].lower()
        context_after = text_str[end_pos:min(len(text_str), end_pos+50)].lower()
        surrounding = text_str[max(0, start_pos-10):end_pos+10]

        # Skip if it's part of a phone number (has parentheses nearby)
        if re.search(r'\(\d{3}\)', surrounding):
            continue

        # Skip if it's in order ID context
        if re.search(r'order\s+(id|number|#)', context_before):
            continue

        # Skip if it's part of a longer number (like part of order ID)
        # Check if there are 6+ digit numbers nearby
        nearby_long_numbers = re.findall(r'\d{6,}', surrounding)
        if nearby_long_numbers:
            # If this 5-digit number is part of a longer sequence, skip it
            continue

        # If it's mentioned near "zip", "address", "location", or city/state, it's likely a zip code
        if any(keyword in context_before or context_after for keyword in ['zip', 'address', 'location', 'city', 'state']):
            return zip_candidate

        # If it appears after a state abbreviation pattern (2 letters + space/number)
        if re.search(r'[A-Z]{2}\s+' + re.escape(zip_candidate), text_str, re.IGNORECASE):
            return zip_candidate

        # If it's a standalone 5-digit number and not in problematic context, consider it
        # But be more conservative - only if it's clearly not part of something else
        if not re.search(r'\d{6,}', surrounding):  # No 6+ digit numbers nearby
            # If there's no clear indication it's NOT a zip code, return it
            # (This catches cases like "78202" mentioned alone)
            return zip_candidate

    return "NA"


In [5]:
def extract_order_id(text):
    """
    Extract order ID from conversation text.
    Order ID is only numbers (no letters).
    Comprehensive pattern matching for various ways order IDs are mentioned.

    Args:
        text: String containing conversation text

    Returns:
        Order ID if found, "NA" otherwise
    """
    if pd.isna(text) or text == "":
        return "NA"

    text_str = str(text)

    # COMPREHENSIVE order ID patterns - try multiple variations
    # Priority order: most specific first

    # Pattern 1: "order ID. It is 1012809669" or "order id: 12345" or "order id is 12345"
    patterns = [
        r'(?:order\s+id\.?\s*(?:it\s+is|is|:)\s*)(\d{6,})',  # "order ID. It is 1012809669" or "order id: 12345" (6+ digits)
        r'(?:order\s+id\.?\s*:?\s*)(\d{6,})',  # "order id: 12345" or "order id 12345" (6+ digits)
        r'(?:order\s+number\.?\s*(?:it\s+is|is|:)?\s*)(\d{6,})',  # "order number: 12345" or "order number 12345"
        r'(?:order\s+#\s*)(\d{6,})',  # "order # 12345"
        r'(?:order\s+)(\d{6,})',  # "order 123456" format (6+ digits)
    ]

    for pattern in patterns:
        matches = list(re.finditer(pattern, text_str, re.IGNORECASE))
        for match in matches:
            order_id = match.group(1)
            # Verify it's only numbers (no letters) and is long enough
            if order_id.isdigit() and len(order_id) >= 6:
                # Skip if it's clearly part of an account ID (has letters nearby)
                start_pos = match.start()
                end_pos = match.end()
                surrounding = text_str[max(0, start_pos-5):end_pos+5]
                # Check if there are letters immediately before or after (account ID pattern)
                if re.search(r'[A-Za-z]\d{6,}|\d{6,}[A-Za-z]', surrounding):
                    continue
                return order_id

    # Pattern 2: Look for numbers that appear right after "order id" on the same or next line
    # This handles cases like:
    # "Do you have an order ID?"
    # "2243746561"
    order_id_context = re.finditer(r'order\s+(?:id|number|#)', text_str, re.IGNORECASE)
    for match in order_id_context:
        # Look for a number within 100 characters after "order id"
        after_text = text_str[match.end():match.end()+100]
        # Find the first standalone number (6+ digits) that appears
        number_match = re.search(r'\b(\d{6,})\b', after_text)
        if number_match:
            order_id = number_match.group(1)
            # Verify it's only numbers and not part of account ID
            if order_id.isdigit():
                # Check if it's part of an account ID (has letters nearby)
                check_text = text_str[match.end():match.end()+number_match.end()+10]
                if not re.search(r'[A-Za-z]\d{6,}|\d{6,}[A-Za-z]', check_text):
                    return order_id

    # Pattern 3: Look for standalone long numeric sequences (9+ digits) that aren't phone numbers
    # Phone numbers have specific formats with parentheses/dashes, so plain long numbers are likely order IDs
    long_number_pattern = r'\b(\d{9,})\b'
    matches = list(re.finditer(long_number_pattern, text_str))

    for match in matches:
        number = match.group(1)
        start_pos = match.start()
        end_pos = match.end()

        # Check context around the number
        context_before = text_str[max(0, start_pos-50):start_pos].lower()
        context_after = text_str[end_pos:min(len(text_str), end_pos+50)].lower()
        surrounding_text = text_str[max(0, start_pos-10):end_pos+10]

        # Skip if it's in a phone number format (has parentheses around area code)
        if re.search(r'\(\d{3}\)', surrounding_text):
            continue

        # Skip if it's in a phone number format with dashes/spaces (XXX-XXX-XXXX)
        if re.search(r'\d{3}[-.\s]\d{3}[-.\s]\d{4}', surrounding_text):
            continue

        # Skip if it looks like a phone number context
        if any(keyword in context_before or context_after for keyword in ['phone', 'call', 'contact', 'telephone']):
            # But allow if it's explicitly in order context
            if 'order' in context_before:
                if number.isdigit():
                    return number
            continue

        # If it's explicitly in order context, return it
        if 'order' in context_before or 'order' in context_after:
            if number.isdigit():
                return number

        # If it's a plain number (no separators) and 9+ digits, it's likely an order ID
        # (phone numbers are typically 10 digits with formatting, order IDs can be longer)
        if number.isdigit() and len(number) >= 9:
            # Double check: if it's exactly 10 digits and not in order context, be cautious
            if len(number) == 10:
                # Only return if there's some indication it's an order ID
                if 'order' in context_before or 'order' in context_after:
                    return number
                # Skip 10-digit numbers that might be phones
                continue
            else:
                # 9 digits or 11+ digits are more likely to be order IDs
                return number

    return "NA"


## Main Processing Function

This function reads the Excel file, extracts all fields, and updates the spreadsheet.


In [6]:
def process_excel_file(input_file, output_file=None):
    """
    Process the Excel file to extract fields from conversation text.

    Args:
        input_file: Path to input Excel file
        output_file: Path to output Excel file (if None, overwrites input file)
    """
    # Read the Excel file
    df = pd.read_excel(input_file)

    print(f"Loaded {len(df)} rows from {input_file}")
    print(f"Columns: {df.columns.tolist()}\n")

    # Extract fields from conversation_text column
    print("Extracting fields...")
    df['email'] = df['conversation_text'].apply(extract_email)
    df['phone'] = df['conversation_text'].apply(extract_phone)
    df['zip_code'] = df['conversation_text'].apply(extract_zip_code)
    df['order_id'] = df['conversation_text'].apply(extract_order_id)

    # Display results
    print("\nExtraction Results:")
    print("=" * 80)
    for idx, row in df.iterrows():
        print(f"\nRow {idx + 1}:")
        print(f"  Email: {row['email']}")
        print(f"  Phone: {row['phone']}")
        print(f"  Zip Code: {row['zip_code']}")
        print(f"  Order ID: {row['order_id']}")

    # Save to output file
    if output_file is None:
        output_file = input_file

    df.to_excel(output_file, index=False)
    print(f"\n\nResults saved to: {output_file}")

    return df


## Execute Extraction

Run the extraction process on the Excel file.


In [7]:
# Specify the input file
input_file = 'GroundTruthregex.xlsx'

# Process the file (you can specify a different output file if needed)
output_file = 'test_GroundTruthregex_final.xlsx'  # Change this to input_file to overwrite

# Process the file
df_results = process_excel_file(input_file, output_file)


Loaded 100 rows from GroundTruthregex.xlsx
Columns: ['conversation_text', 'convo_id', 'email', 'phone', 'order_id', 'zip_code']

Extracting fields...

Extraction Results:

Row 1:
  Email: cm442144@email.com
  Phone: NA
  Zip Code: NA
  Order ID: 1012809669

Row 2:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 8873864352

Row 3:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 5189186577

Row 4:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 3278902139

Row 5:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 7589935390

Row 6:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 0988234608

Row 7:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 5628882649

Row 8:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 3536918602

Row 9:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: NA

Row 10:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: 5827805604

Row 11:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order ID: NA

Row 12:
  Email: NA
  Phone: NA
  Zip Code: NA
  Order I

## View Results Summary

Display a summary of the extracted data.


In [8]:
# Display summary
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"\nTotal rows processed: {len(df_results)}")
print(f"\nFields extracted:")
print(f"  Emails found: {len(df_results[df_results['email'] != 'NA'])}")
print(f"  Phones found: {len(df_results[df_results['phone'] != 'NA'])}")
print(f"  Zip codes found: {len(df_results[df_results['zip_code'] != 'NA'])}")
print(f"  Order IDs found: {len(df_results[df_results['order_id'] != 'NA'])}")

# Display the updated dataframe
print("\n" + "=" * 80)
print("UPDATED DATAFRAME")
print("=" * 80)
# Use display() if in Jupyter, otherwise print
try:
    from IPython.display import display
    display(df_results[['convo_id', 'email', 'phone', 'zip_code', 'order_id']])
except:
    print(df_results[['convo_id', 'email', 'phone', 'zip_code', 'order_id']].to_string())



SUMMARY

Total rows processed: 100

Fields extracted:
  Emails found: 22
  Phones found: 6
  Zip codes found: 11
  Order IDs found: 36

UPDATED DATAFRAME


,convo_id,email,phone,zip_code,order_id
0,7807,cm442144@email.com,NA,NA,1012809669
1,2776,NA,NA,NA,8873864352
2,219,NA,NA,NA,5189186577
3,9104,NA,NA,NA,3278902139
4,2109,NA,NA,NA,7589935390
...,...,...,...,...,...
95,6750,NA,NA,NA,NA
96,7454,NA,NA,NA,NA
97,4657,NA,NA,NA,NA
98,3583,NA,NA,NA,NA
